In [2]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession


#Creating the Spark Session:
spark = SparkSession.builder.appName("CrashesAnalysis").getOrCreate()

In [8]:
#Reading the CSVs in DataFrames
df_charges = spark.read.csv("/home/ishaan25/Documents/BGC_UseCase/Data/Charges_use.csv",header=True)
df_damages = spark.read.csv("/home/ishaan25/Documents/BGC_UseCase/Data/Damages_use.csv",header=True)
df_endorse = spark.read.csv("/home/ishaan25/Documents/BGC_UseCase/Data/Endorse_use.csv",header=True)
df_primary_person = spark.read.csv("/home/ishaan25/Documents/BGC_UseCase/Data/Primary_Person_use.csv",header=True)
df_restrict = spark.read.csv("/home/ishaan25/Documents/BGC_UseCase/Data/Restrict_use.csv",header=True)
df_units = spark.read.csv("/home/ishaan25/Documents/BGC_UseCase/Data/Units_use.csv",header=True)

In [190]:
df_charges.show(10,False)
#df_charges.where("charge like '%SPEED%'").select(F.col('CHARGE')).distinct().show(50,False)

+--------+--------+--------+----------------------------------------+------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|CHARGE                                  |CITATION_NBR|
+--------+--------+--------+----------------------------------------+------------+
|14768622|1       |1       |DRIVING WHILE INTOXICATED               |null        |
|14838637|1       |1       |DWI                                     |1600000015  |
|14838641|1       |1       |RAN RED LIGHT SOLID (TC 544.007)        |L20440      |
|14838641|2       |1       |NO DRIVER'S LICENSE (TC521.025)         |L23141      |
|14838668|1       |1       |DRIVING WHILE INTOXICATED               |TX4IC50SRJD3|
|14838669|2       |1       |DWI W/BAC >.015                         |2015-000006 |
|14838670|1       |1       |DRIVING WHILE INTOXICATED               |2016-000003 |
|14838685|1       |1       |FAILED TO DRIVE SINGLE LANE, NOL, NO INS|138434825   |
|14838693|1       |1       |DRIVING WHILE INTOXICATED               |TX4IC60UKQND|
|148

In [9]:
df_damages.show(10,False)

+--------+---------------------------------+
|CRASH_ID|DAMAGED_PROPERTY                 |
+--------+---------------------------------+
|14768622|MAILBOX                          |
|14768622|YARD, GRASS                      |
|14838668|GUARDRAIL                        |
|14838685|ROAD SIGN                        |
|14838693|2009 MAZDA 3                     |
|14838834|CHAIN LINK FENCE                 |
|14838841|WOODED POLE ON SOUTH SIDE OF LOOP|
|14838842|CITY SIGN FOR TURN LANES         |
|14838877|FENCE-CHAIN LINK                 |
|14838977|LANDSCAPING AND METAL SIGN       |
+--------+---------------------------------+
only showing top 10 rows



In [191]:
df_endorse.select(F.col('DRVR_LIC_ENDORS_ID')).distinct().show(100,False)
#df_endorse.select(F.col('crash_id')).distinct().count()

+-------------------------------------+
|DRVR_LIC_ENDORS_ID                   |
+-------------------------------------+
|SCHOOL BUS                           |
|OTHER/OUT OF STATE                   |
|UNKNOWN                              |
|HAZARDOUS MATERIALS                  |
|UNLICENSED                           |
|TANK VEHICLE                         |
|NONE                                 |
|TANK VEHICLE WITH HAZARDOUS MATERIALS|
|PASSENGER                            |
|DOUBLE/TRIPLE TRAILER                |
+-------------------------------------+



In [189]:
df_primary_person.show(2,False)
#df_primary_person.select(F.col('crash_id')).distinct().count()

+--------+--------+--------+------------+------------------+-----------------+--------+-----------------+------------+------------+-------------------+-----------------+--------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+----------------+-----------------+------------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|PRSN_TYPE_ID|PRSN_OCCPNT_POS_ID|PRSN_INJRY_SEV_ID|PRSN_AGE|PRSN_ETHNICITY_ID|PRSN_GNDR_ID|PRSN_EJCT_ID|PRSN_REST_ID       |PRSN_AIRBAG_ID   |PRSN_HELMET_ID|PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT|PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|PRSN_DEATH_TIME|INCAP_INJRY_CNT|NONINCAP_INJRY_CNT|POSS_INJRY_CNT|NON_INJRY_CNT|UNKN_INJRY_CNT|TOT_INJRY_CNT|DEATH_CNT|DRVR_LIC_TYPE_ID|DRVR_LIC_STATE_ID|DRVR_LIC_CLS_ID   |DRVR_ZIP|
+--------+--

In [167]:
df_restrict.show(10,False)

+--------+--------+----------------------+
|CRASH_ID|UNIT_NBR|DRVR_LIC_RESTRIC_ID   |
+--------+--------+----------------------+
|14768622|1       |NONE                  |
|14838637|1       |OTHER/OUT OF STATE    |
|14838637|2       |NONE                  |
|14838641|1       |WITH CORRECTIVE LENSES|
|14838641|2       |UNLICENSED            |
|14838668|1       |NONE                  |
|14838669|1       |NONE                  |
|14838669|2       |WITH CORRECTIVE LENSES|
|14838670|1       |NONE                  |
|14838685|1       |UNLICENSED            |
+--------+--------+----------------------+
only showing top 10 rows



In [118]:
#df_units.select(F.col('CONTRIB_FACTR_1_ID')).distinct().show(200,False)

df_units.where("CONTRIB_FACTR_1_ID in ('UNDER INFLUENCE - ALCOHOL','HAD BEEN DRINKING')").select(F.col("crash_id"),F.col("CONTRIB_FACTR_1_ID")).show(5,False)

+--------+-------------------------+
|crash_id|CONTRIB_FACTR_1_ID       |
+--------+-------------------------+
|14768622|UNDER INFLUENCE - ALCOHOL|
|14838637|HAD BEEN DRINKING        |
|14838668|UNDER INFLUENCE - ALCOHOL|
|14838670|HAD BEEN DRINKING        |
|14838693|UNDER INFLUENCE - ALCOHOL|
+--------+-------------------------+
only showing top 5 rows



In [17]:
#Analytics 1: Find the number of crashes (accidents) in which number of persons killed are male?
import pyspark.sql.functions as F
df_primary_person.select(F.col("PRSN_INJRY_SEV_ID")).distinct().show(20,False)

+-------------------------+
|PRSN_INJRY_SEV_ID        |
+-------------------------+
|NA                       |
|KILLED                   |
|UNKNOWN                  |
|NON-INCAPACITATING INJURY|
|NOT INJURED              |
|POSSIBLE INJURY          |
|INCAPACITATING INJURY    |
+-------------------------+



In [28]:
#Ans 1:
#No of Acc = 182, Distinct Acc = 180 (In 2 Acc, 2-2 Males died in crash_id '15429998','15379024')
df_primary_person.where("PRSN_INJRY_SEV_ID = 'KILLED' and PRSN_GNDR_ID = 'MALE'").select(F.col("CRASH_ID")).distinct().count()

180

In [25]:
df_primary_person.registerTempTable("person")
#where("PRSN_INJRY_SEV_ID = 'KILLED' and PRSN_GNDR_ID = 'MALE'").groupby(F.col("CRASH_ID")).having.distinct().show(100,False)

In [26]:
spark.sql("select crash_id,count(1) from person where PRSN_INJRY_SEV_ID = 'KILLED' and PRSN_GNDR_ID = 'MALE' group by crash_id having count(1)>1").show(10,False)

+--------+--------+
|crash_id|count(1)|
+--------+--------+
|15429998|2       |
|15379024|2       |
+--------+--------+



In [27]:
df_primary_person.where("crash_id in ('15429998','15379024')").show(10,False)
PRSN_INJRY_SEV_ID

+--------+--------+--------+------------+-------------------------------------------------+-------------------------+--------+-----------------+------------+--------------+-------------------+--------------+--------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+----------------+-----------------+------------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|PRSN_TYPE_ID|PRSN_OCCPNT_POS_ID                               |PRSN_INJRY_SEV_ID        |PRSN_AGE|PRSN_ETHNICITY_ID|PRSN_GNDR_ID|PRSN_EJCT_ID  |PRSN_REST_ID       |PRSN_AIRBAG_ID|PRSN_HELMET_ID|PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT|PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|PRSN_DEATH_TIME|INCAP_INJRY_CNT|NONINCAP_INJRY_CNT|POSS_INJRY_CNT|NON_INJRY_CNT|UNKN_INJRY_CNT|TOT_INJRY_CNT|DEATH_CNT|

In [41]:
#Analysis 2: How many two wheelers are booked for crashes:

df_units.where("VEH_BODY_STYL_ID in ('MOTORCYCLE','POLICE MOTORCYCLE')").select(F.col("CRASH_ID")).distinct().count()

757

In [42]:
#Analysis 3: Which state has highest number of accidents in which females are involved? 
df_primary_person.where("PRSN_GNDR_ID = 'FEMALE'").groupby(F.col("DRVR_LIC_STATE_ID")).count().distinct().sort(F.col('count').desc()).show(1,False)
#select state_id,count(1) from person where person_dender = female group by LIC_state order by count(1) desc limit 1

+-----------------+-----+
|DRVR_LIC_STATE_ID|count|
+-----------------+-----+
|Texas            |53319|
+-----------------+-----+
only showing top 1 row



In [82]:
#Analysis 4: Which are the Top 5th to 15th VEH_MAKE_IDs that 
#contribute to a largest number of injuries including death

from pyspark.sql.window import Window

df_req_units = df_units.select(F.col("VEH_MAKE_ID"),F.col("CRASH_ID"))
#df_req_units.show(5,False)
df_injured_person = df_primary_person.where("PRSN_INJRY_SEV_ID in ('KILLED','NON-INCAPACITATING INJURY','INCAPACITATING INJURY','POSSIBLE INJURY')").select(F.col('CRASH_ID')).distinct()
#df_injured_person.show(5,False)
join_cond = [df_injured_person['crash_id'] == df_req_units['crash_id']]
df_units_with_injury = df_req_units.join(df_injured_person, on = join_cond, how='inner').drop(df_req_units['crash_id'])
df_req = df_units_with_injury.groupby(F.col("VEH_MAKE_ID")).count().sort(F.col('count').desc())
df_req_rnk = df_req.withColumn("crash_cnt",F.row_number().over(Window.orderBy(F.col('count').desc())))
df_req_rnk.where("crash_cnt between 5 and 15").select(F.col('VEH_MAKE_ID')).show(15,False)

#df_units_with_injury.show(50,False)


+------------+
|VEH_MAKE_ID |
+------------+
|NISSAN      |
|HONDA       |
|NA          |
|GMC         |
|JEEP        |
|HYUNDAI     |
|KIA         |
|CHRYSLER    |
|FREIGHTLINER|
|MAZDA       |
|LEXUS       |
+------------+



In [130]:
#Analysis 5: For all the body styles involved in crashes, 
#mention the top ethnic user group of each unique body style  
windowDept = Window.partitionBy(F.col('VEH_BODY_STYL_ID')).orderBy(F.col("count").desc())

df_person_with_ethnicity = df_primary_person.select(F.col('CRASH_ID'),F.col('PRSN_ETHNICITY_ID')).distinct()
df_req_units = df_units.select(F.col("VEH_BODY_STYL_ID"),F.col("CRASH_ID"))
join_cond = [df_person_with_ethnicity['crash_id'] == df_req_units['crash_id']]
df_units_with_ethnicity = df_req_units.join(df_person_with_ethnicity, on = join_cond, how='inner').drop(df_req_units['crash_id'])
df_rnked = df_units_with_ethnicity.groupby(F.col('VEH_BODY_STYL_ID'),F.col('PRSN_ETHNICITY_ID')).count().sort(F.col('VEH_BODY_STYL_ID'),F.col('count').desc())
df_req = df_rnked.withColumn("body_type_rnk",F.row_number().over(windowDept))
#df_req.show(100,False)
df_res = df_req.where("body_type_rnk = 1").select(F.col('VEH_BODY_STYL_ID'),F.col('PRSN_ETHNICITY_ID'))
df_res.show(50,False)

+---------------------------------+-----------------+
|VEH_BODY_STYL_ID                 |PRSN_ETHNICITY_ID|
+---------------------------------+-----------------+
|BUS                              |HISPANIC         |
|NA                               |WHITE            |
|VAN                              |WHITE            |
|PICKUP                           |WHITE            |
|SPORT UTILITY VEHICLE            |WHITE            |
|PASSENGER CAR, 4-DOOR            |WHITE            |
|FIRE TRUCK                       |WHITE            |
|TRUCK                            |WHITE            |
|UNKNOWN                          |UNKNOWN          |
|AMBULANCE                        |WHITE            |
|POLICE CAR/TRUCK                 |WHITE            |
|MOTORCYCLE                       |WHITE            |
|YELLOW SCHOOL BUS                |WHITE            |
|POLICE MOTORCYCLE                |HISPANIC         |
|PASSENGER CAR, 2-DOOR            |WHITE            |
|TRUCK TRACTOR              

In [160]:
#Analysis 6: Among the crashed cars, what are the Top 5 Zip Codes with highest number crashes 
#with alcohols as the contributing factor to a crash (Use Driver Zip Code)

df_alc_crashes = df_units.where("VEH_BODY_STYL_ID in ('PASSENGER CAR, 2-DOOR','POLICE CAR/TRUCK','PASSENGER CAR, 4-DOOR','SPORT UTILITY VEHICLE')").where("CONTRIB_FACTR_1_ID in ('UNDER INFLUENCE - ALCOHOL','HAD BEEN DRINKING') or CONTRIB_FACTR_2_ID in ('UNDER INFLUENCE - ALCOHOL','HAD BEEN DRINKING') or CONTRIB_FACTR_P1_ID in ('UNDER INFLUENCE - ALCOHOL','HAD BEEN DRINKING')").select(F.col("crash_id"),F.col('VEH_BODY_STYL_ID'))
#df_alc_crashes.show(10,False)
df_person_with_zip = df_primary_person.select(F.col("crash_id"),F.col('DRVR_ZIP'))
join_cond = [df_alc_crashes['crash_id'] == df_person_with_zip['crash_id']]
df_req = df_alc_crashes.join(df_person_with_zip, on = join_cond, how='inner').drop(df_person_with_zip['crash_id'])
#df_req.show(15,False)
df_res = df_req.groupby(F.col('DRVR_ZIP')).count()
df_res.sort(F.col('count').desc()).show(5,False)

#df_primary_person.select(F.col('PRSN_ALC_RSLT_ID')).distinct().show(10,False)

+--------+-----+
|DRVR_ZIP|count|
+--------+-----+
|null    |347  |
|78521   |75   |
|75067   |57   |
|78753   |55   |
|75070   |52   |
+--------+-----+
only showing top 5 rows



In [180]:
#Analysis 7: Count of Distinct Crash IDs where No Damaged Property was observed 
#and Damage Level (VEH_DMAG_SCL~) is above 4 
#and car avails Insurance

df_damage_cid = df_damages.select(F.col('crash_id')).distinct()
df_unit_dmg_lvl = df_units.where("VEH_DMAG_SCL_1_ID in('DAMAGED 5','DAMAGED 6','DAMAGED 7 HIGHEST') or VEH_DMAG_SCL_2_ID in('DAMAGED 5','DAMAGED 6','DAMAGED 7 HIGHEST')")
df_unit_with_insurance = df_unit_dmg_lvl.where("FIN_RESP_TYPE_ID like '%INSURANCE%'").select(F.col('crash_id'),F.col('VEH_DMAG_SCL_1_ID'),F.col('VEH_DMAG_SCL_2_ID'),F.col('FIN_RESP_TYPE_ID'))
join_cond = [df_unit_with_insurance['crash_id'] == df_damage_cid['crash_id']]
df_req = df_unit_with_insurance.join(df_damage_cid, on = join_cond, how='left_anti').distinct()#.drop(df_person_with_zip['crash_id'])
df_req.select(F.col('crash_id')).distinct().count()

#df_damage_cid.where("crash_id in ('14839240','14839343','14839510','14840005')").show()

8849

In [174]:
df_units.where("FIN_RESP_TYPE_ID like '%INSURANCE%'").select(F.col('FIN_RESP_TYPE_ID')).distinct().show(10,False)

+-----------------------------+
|FIN_RESP_TYPE_ID             |
+-----------------------------+
|INSURANCE BINDER             |
|LIABILITY INSURANCE POLICY   |
|CERTIFICATE OF SELF-INSURANCE|
|PROOF OF LIABILITY INSURANCE |
+-----------------------------+



In [218]:
#Analysis 8: Determine the Top 5 Vehicle Makes where drivers are charged with speeding related offences, 
#has licensed Drivers, 
#uses top 10 used vehicle colours 
#and has car licensed with the Top 25 states with highest number of offences (to be deduced from the data)

#Getting crashes where charge was speeding related:
df_speeding_charge = df_charges.where("charge like '%SPEED%'").select(F.col('crash_id'),F.col('CHARGE'))

#Getting the top 25 states with highest number of offences:
df_top_state = df_primary_person.groupby(F.col('DRVR_LIC_STATE_ID')).count()
df_rnk_state = df_top_state.withColumn('state_rnk',F.row_number().over(Window.orderBy(F.col('count').desc())))
df_rnk_state_req = df_rnk_state.where("state_rnk <= 25").select(F.col('DRVR_LIC_STATE_ID'))
#df_rnk_state_req.show(10,False)

#Getting Licensed Drivers:
df_lic_drvr = df_primary_person.where("DRVR_LIC_TYPE_ID in ('COMMERCIAL DRIVER LIC.','DRIVER LICENSE')").select(F.col('crash_id'),F.col('DRVR_LIC_STATE_ID'),F.col('DRVR_LIC_TYPE_ID'))

#Getting Licensed Drivers those have car licensed with the Top 25 states with highest number of offences:
df_lic_drvr_top_state = df_lic_drvr.join(df_rnk_state_req, on = [df_lic_drvr['DRVR_LIC_STATE_ID'] == df_rnk_state_req['DRVR_LIC_STATE_ID']],how = 'inner').drop(df_rnk_state_req['DRVR_LIC_STATE_ID'])
#df_lic_drvr_top_state.show(10,False)

#Getting Licensed Drivers from Top states with speeding related charges:
drvr_speed_cond = [df_lic_drvr_top_state['crash_id'] == df_speeding_charge['crash_id']]
df_lic_drvr_speeding = df_lic_drvr_top_state.join(df_speeding_charge, on = drvr_speed_cond, how = 'inner').drop(df_speeding_charge['crash_id'])
#df_lic_drvr_speeding.show(15,False)

#Getting the top 10 used vehicles colours:
df_top_clr_vhcl = df_units.groupby(F.col('VEH_COLOR_ID')).count()
df_rnk_clr = df_top_clr_vhcl.withColumn('clr_rnk',F.row_number().over(Window.orderBy(F.col('count').desc())))
df_rnk_clr_req = df_rnk_clr.where("clr_rnk <= 10").select(F.col('VEH_COLOR_ID'))
#df_rnk_clr_req.show(10,False)

#Getting crash_ids where the colour of car was in top 10 used vehicles colours:
df_units_req = df_units.join(df_rnk_clr_req, on = [df_units['VEH_COLOR_ID'] == df_rnk_clr_req['VEH_COLOR_ID']], how = 'inner').drop(df_rnk_clr_req['VEH_COLOR_ID'])
df_units_top_clr = df_units_req.select(F.col('crash_id'),F.col('VEH_COLOR_ID'),F.col('VEH_MAKE_ID'))
#df_units_top_clr.show(10,False)

#Getting licensed speedy drivers from Top states driving top coloured cars:
drvr_clr_cond = [df_lic_drvr_speeding['crash_id'] == df_units_top_clr['crash_id']]
df_req = df_lic_drvr_speeding.join(df_units_top_clr, on = drvr_clr_cond,how = 'inner')

#Getting the Top Vehicle Makers:
df_veh_req = df_req.groupby(F.col('VEH_MAKE_ID')).count()
df_veh_req_rnk = df_veh_req.withColumn('veh_rnk',F.row_number().over(Window.orderBy(F.col('count').desc())))
df_veh_req_rnk.where("veh_rnk <= 5").select(F.col('VEH_MAKE_ID')).show(10,False)





+-----------+
|VEH_MAKE_ID|
+-----------+
|FORD       |
|CHEVROLET  |
|TOYOTA     |
|DODGE      |
|NISSAN     |
+-----------+



In [215]:
#Getting the top 10 used vehicals colours:
df_top_clr_vhcl = df_units.groupby(F.col('VEH_COLOR_ID')).count()
df_rnk_clr = df_top_clr_vhcl.withColumn('clr_rnk',F.row_number().over(Window.orderBy(F.col('count').desc())))
df_rnk_clr_req = df_rnk_clr.where("clr_rnk <= 10").select(F.col('VEH_COLOR_ID'))
#df_rnk_clr_req.show(10,False)

#Getting crash_ids where the colour of car was in top 10 used vehicals colours:
df_units_req = df_units.join(df_rnk_clr_req, on = [df_units['VEH_COLOR_ID'] == df_rnk_clr_req['VEH_COLOR_ID']], how = 'inner').drop(df_rnk_clr_req['VEH_COLOR_ID'])
df_units_top_clr = df_units_req.select(F.col('crash_id'),F.col('VEH_COLOR_ID'))
#df_units_top_clr.show(10,False)

df_units.select('VEH_MAKE_ID').distinct().show(25,False)

+-----------------------------+
|VEH_MAKE_ID                  |
+-----------------------------+
|UNITED EXPRESS LINE INC      |
|UTILIMASTER                  |
|AMERICAN IRON HORSE          |
|ACURA                        |
|NOVA                         |
|FRUEHAUF                     |
|WHITE                        |
|MOTOR COACH MND INC          |
|MCLAREN                      |
|PIERCE                       |
|PORSCHE                      |
|WHITEGMC                     |
|FREIGHTLINER                 |
|BUELL                        |
|STERLING                     |
|HYUNDAI                      |
|INTERNATIONAL                |
|PETER PIRSCH & SONS          |
|FIAT                         |
|GILLIG                       |
|NA                           |
|MIDLAND MANUFACTURING LIMITED|
|DATSUN                       |
|CAN-AM                       |
|HYUNDAI STEEL INDUSTRIES     |
+-----------------------------+
only showing top 25 rows

